In [ ]:
# Imports
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Custom dataset to train an NN with simple data and target tensors.
class CustomTensorDataset(Dataset):
    def __init__(self, data_tensor, target_tensor):
        self.data = data_tensor
        self.target = target_tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.target[index]

In [ ]:
# Example tensors for learning addition.

def compute_polynomial(x):
    # Given a value of x, compute a degree-2 polynomial with three fixed coefficients.
    c2 = -7
    c1 = 2
    c0 = 14
    return c2*x[0]**2 + c1*x[0]**1 + c0

# Data tensor is 100,000 values of x.
# Target tensor is the value of the function above for each x.
train_data_tensor = torch.rand(100000, 1)
train_target_tensor = torch.tensor([compute_polynomial(x) for x in train_data_tensor])

# Create dataset instance
train_dataset = CustomTensorDataset(train_data_tensor, train_target_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# And we do the same to create a separate test dataloader.
test_data_tensor = torch.rand(100000, 1)
test_target_tensor = torch.tensor([compute_polynomial(x) for x in test_data_tensor])

test_dataset = CustomTensorDataset(test_data_tensor, test_target_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
# Get training device.
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Define our NN structure to learn addition of 10 integers.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1, 5),
            nn.Sigmoid(),
            nn.Linear(5, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# Construct and move NN to device.
model = NeuralNetwork().to(device)
print(model)

In [ ]:
# Training params and loss function.
learning_rate = 1e-3
batch_size = 64
epochs = 10

# Since we're estimating a function (sum of all elements in input vector), mean squared error seems best.
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
# Try one more.
x = torch.rand(1,1)[0]
print(x)
print(compute_polynomial(x))

print(model(x))

In [ ]:
foo = torch.arange(-5000,5000,0.1)
print(foo)
print(len(foo))